In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# для построения суррогатных моделей
from sklearn.base import BaseEstimator

# LS - The Least square model
from smt.surrogate_models import LS, KRG

# RBF из SMT не подключается (из другой библиотеки - всё норм) [проблема именно на домашнем компьютере]
from smt.surrogate_models import RBF

# для обрабтки входящих массивов
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [22]:
class DISTRIBUTION_PREDICTION(BaseEstimator):
    all_surrogate_models = {
        # все рассматриваемые суррогатные модели. Можно расширить этот список
        "LS" : LS,
        "KRG" : KRG,
        "RBF" : RBF
    }
    
    def __init__(self, surrogate_model_name="LS", Npc=3, n_parts=1, parts = None, 
                 kw_args={'print_global':False}, hello_words=False):
        """
        модель, предсказывающая распределённый по крылу параметр
        поля мараметра каждого прецидента вытягиваются в 1D-вектор и затем комбинируются в мутрицу полей
        модель делает сингулярное разложение матрицы полей и выбирает Npc главных компонент. 
        Затем, суррогатные модели обучаются предсказывать коэффициенты, с которыми надо складывать
        выбранные главные компоненты, чтобы предсказать исходное распределение параметра по крылу 
        
        Input:
            surrogate_model_name - str - название используемой суррогатной модели. Пока доступны "LS", "KRG", "RBF"
            Npc - int - число используемых главных компонент
            n_parts - int - на сколько частей делить расчётную область. 
                            Для каждой части будет построена своя суррогатная модель.
                            Есть два крайних случая: 
                                n_parts=1, когда строится одна СМ над всем крылом
                                n_marts=число точек в расчётной сетке, тогда для каждой расчётной точки 
                                будет построена своя суррогатная модель
            parts - list(list(inds)) - Каждый элемент списка parts - это список, содержащий индексы, 
                                       относящиеся к какой-то части крыла.
                                       Если parts=`None`, то индексы распределяются равномерно, согласно n_parts
            kw_args -  словарь {'arg_name':arg_value} - словарь параметров для используемой суррогатной модели
            hello_words - bool - флаг. Надо ли выводить информацию при работе
        
        
        """
        assert parts == None or n_parts == len(parts), \
            f"n_parts={n_parts} не совпадает с количеством частей, указанных в parts. \
            Сделайте partas == None или n_parts == len(parts)"
        assert surrogate_model_name in self.all_surrogate_models.keys(), \
            'SM-name must be on of the following: {}'.format(self.all_surrogate_models.keys())
        
        if hello_words:
            print(f"{surrogate_model_name}, Npc={Npc} init")
        
        self.Npc = Npc    
        self.n_parts = n_parts
        self.parts = parts
        self.surrogate_model_name = surrogate_model_name
        self.kw_args = kw_args
        self.hello_words = hello_words
        
    def partition(self):
        """
        Создадим список списков индексов.
        По итогу работы функции вернётся список parts. 
        Каждый элемент списка parts - это список, содержащий индексы, относящиеся к какой-то части крыла.
        
        В случае, если n_parts=1, функция вернёт список всех точек крыла
        """
        n_elements_in_part = self.Nrows // self.n_parts
        parts = [np.arange(i*n_elements_in_part,(i+1)*n_elements_in_part) for i in range(self.n_parts-1)]
        parts.append(np.arange((self.n_parts-1)*n_elements_in_part,self.Nrows))
        return parts
        
    def get_basis_and_trainCoefficients_for_part(self, target_part):
        """
        Input:
            target_part - np.array (Nrows_part,Nsamples)
        Output:
            basis - np.array(Nrows_part,Npc)
            k - np.array(Nsamples,Npc)
        
        По итогу работы это функции вернутся матрица basis (Nrows_part,Npc), каждой столбец которой это главные 
        компнонеты, которые будут использоваться при предсказании, и матрица k (Nsamples, Npc), каждая строка 
        которой содержит коэффициенты разложения по главным компонентам.  
        """
        U, s, Vt = np.linalg.svd(target_part, full_matrices=False)
                
        # Npc базисных столбцов
        basis = U[:,:self.Npc]*s[:self.Npc]
        
        # теперь нужно получить Nsamples наборов из Npc коэффициентов для каждого из Nsamples режимов
        k = Vt[:self.Npc].T
        
        return basis, k
    
    def train_model_for_part(self, xtrain, ytrain):
        """
        Input:
            xtrain - np.array(Nsamples, 2) - вектор из пар (M, alpha)
            ytrain - np.array(Nsamples, Npc) - векотор целевых переменных-коэфициентов при главных компонентах
        Output:
            predictor - обученная суррогатная модель для части кр
        """        
        predictor = self.surrogate_model(**self.kw_args)
        predictor.set_training_values(xt=xtrain,yt=ytrain)
        predictor.train()
        
        return predictor
    
    def fit(self, regimes, target_T):
        """
        Обучение предиктора для предсказания по режиму потока распределения целевого параметра по крылу
        Input:
            regimes - DataFrame (Nsamples, 2) - столбец режимов. Каждая строчка содержит пару (M, alpha)
            target_T - DataFrame (Nsamples, Nrows) - каждая строка - это распределения целевого параметра 
                                                 по поверхности крыла при конкретном режиме
        Output:
            ничего. Просто обучает предиктор
        """
        regimes, target_T = check_X_y(regimes, target_T, multi_output=True)
        # regimes, target_T уже сконвертированы в np.array 
        
        target = target_T.T # target=DF(Nrows, Nsamples)
        
        self.surrogate_model = self.all_surrogate_models[self.surrogate_model_name]
        self.Predictors_n_parts = [] # список предикторов для каждой части крыла
                                     # каждый предиктор является суррогатной моделью
        self.basis_n_parts = [] # список базисов из главных компонент для каждой части крыла
        self.Nrows, self.Nsamples = target.shape
        
        if self.parts == None:
            self.parts = self.partition()
            
        
        for part in self.parts:
            target_part = target[part]
            
            basis, ytrain = self.get_basis_and_trainCoefficients_for_part(target_part)
            self.basis_n_parts.append(basis)
            
            predictor = self.train_model_for_part(xtrain=regimes, ytrain=ytrain)
            self.Predictors_n_parts.append(predictor)        
        return
    
    def predict_part(self, regimes, part_num):
        """
        Предсказание распределений целевой переменной на части крыла по известным режимам обтекания.
        Input:
            regimes - DataFrame (Npoints, 2) - столбец режимов. Каждая строчка содержит пару (M, alpha)
            part_num - int - номер части крыла, на которой надо сделать предсказание
        Output:
            target_part - DataFrame (Nrows_part,Npoints) - матрица распределений целевого значения
                                                           по части крыла. каждый столбец соответствует 
                                                           своему режиму.
        """        
        xtest = np.array(regimes)
        coeficients = (self.Predictors_n_parts[part_num]).predict_values(xtest)
        
        basis = self.basis_n_parts[part_num]
        prediction = basis @ coeficients.T
        
        target_part = pd.DataFrame(prediction, index=self.parts[part_num])
        return target_part
    
    def predict(self, regimes):
        """
        Предсказание распределения целевого параметра по всему крылу по известным режимам обтекания
        Input:
            regimes - DataFrame (Npoints, 2) - столбец режимов. Каждая строчка содержит пару (M, alpha)
        Output:
            target - DataFrame(Npoint, Nrows) - матрица распределений целевого параметра по всему крылу.
                                              Каждый столбец соответствует своему режиму.
        """
        regimes = check_array(regimes)
        list_of_target_parts = []
        for i in range(len(self.parts)):
            target_part = self.predict_part(regimes, i)
            list_of_target_parts.append(target_part)
            
        target = pd.concat(list_of_target_parts, axis=0)
        return target.T

In [23]:
class SOLVER_INTEGRALVALUES(BaseEstimator):
    all_surrogate_models = {
        "LS" : LS,
        "KRG" : KRG,
        "RBF" : RBF
    }
    
    def __init__(self, surrogate_model_name="LS", kw_args={'print_global':False}, hello_words=False):
        """
        модель, предсказывающая 1, 2, или n параметров. 
        обыкновенная суррогатная модель
        
        Input:
            surrogate_model_name -  str - название используемой суррогатной модели. Пока доступны "LS", "KRG", "RBF"
            kw_args -  словарь {'arg_name':arg_value} - словарь параметров для используемой суррогатной модели
            hello_words - bool - флаг. Надо ли выводить информацию при работеs
        
        """
        assert surrogate_model_name in self.all_surrogate_models.keys(), \
            'SM-name must be on of the following: {}'.format(self.all_surrogate_models.keys())

        if hello_words:
            print(f"{surrogate_model_name} init")
        
        self.surrogate_model_name = surrogate_model_name
        self.kw_args = kw_args
        self.hello_words = hello_words
        
    def fit(self, regimes, target):
        """
        Обучение предиктора на предсказание по режиму потока целевой интегральной величины
        Input:
            regimes - DataFrame (Nsamples, 2) - столбец режимов. Каждая строчка содержит пару (M, alpha)
            target - DataFrame (Nsamples, Nrows) - каждая строка - это значения целевых интегральных велечин
        Output:
            ничего. Просто обучает предиктор
        """
        regimes, target = check_X_y(regimes, target, multi_output=True)
        # regimes, target уже сконвертированы в np.array 
        
        self.surrogate_model = self.all_surrogate_models[self.surrogate_model_name](**self.kw_args)
        
        self.surrogate_model.set_training_values(xt=regimes, yt=target)
        self.surrogate_model.train()   
        return
    
    def predict(self, regimes):
        """
        Предсказание целевых интегральных величин по известным режимам обтекания
        Input:
            regimes - DataFrame (Nsamples, 2) - столбец режимов. Каждая строчка содержит пару (M, alpha)
        Output:
            target - DataFrame(Nsamples, Nrows) - матрица целевых значений.
                                                  Каждый столбец соответствует своему режиму.
        """
        regimes = check_array(regimes)
    
        target = (self.surrogate_model).predict_values(regimes)
        return target